In [ ]:
# argument user and restaurant reviews and category information
# summerize them into user preferences and restaurant features and basic information

def genai_argument_method_v1(original_data, client, system_role_content, prompt, genai_model="gpt-3.5-turbo"):
    
    original_cols = list(original_data.columns)
    id_column = [col for col in original_cols if 'id' in col.lower()][0]
    text_column = [col for col in original_cols if 'text' in col.lower() or 'review' in col.lower() or 'categor' in col.lower()][0] # user_concatenated_reviews_text ...
    new_cols = list(original_data.columns)
    new_cols.extend(["system_role_content", "prompt", f"{text_column}_argumented_text_result"])
    argumented_results = pd.DataFrame(columns=new_cols)
    filtered_original_data = original_data.copy()
    
    file_path = "_".join(original_cols) + "_with_argumented_results.csv"
    if not os.path.exists(file_path):
        empty_df = pd.DataFrame(columns=new_cols)
        empty_df.to_csv(file_path, encoding='utf-8', index=False)
    else:
        results_df = pd.read_csv(file_path)
        if len(results_df) == len(original_data):
            return "You already have full argumented results of this original data, please check !"
        else:
            original_ids = set(original_data[id_column].values)
            results_df_ids = set(results_df[id_column].values)
            remain_ids = original_ids - results_df_ids
            filtered_original_data = original_data[original_data[id_column].isin(remain_ids)]
            filtered_original_data.reset_index(drop=True, inplace=True)
    
    for i, row in filtered_original_data.iterrows():

        entity_id, original_text = list(row)
        index = original_data[id_column].index[original_data[id_column] == entity_id].tolist()[0]
        content_for_model = prompt + "\n\n" + original_text
        
        print(f"Processing {index+1}th data, with entity id = {entity_id}")

        completion = client.chat.completions.create(
          model=genai_model,
          messages=[
            {"role": "system", "content": system_role_content},
            {"role": "user", "content": content_for_model}
          ]
        )

        argumented_text_result = completion.choices[0].message.content
#         argumented_text_result = "argumented text test"
        temp_df = pd.DataFrame([[entity_id, original_text, system_role_content, prompt
                                 , argumented_text_result]], columns=new_cols)
        temp_df.to_csv(file_path, mode='a', encoding='utf-8', header=False, index=False)
        argumented_results = pd.concat([argumented_results, temp_df], ignore_index=True)

    return argumented_results